In [ ]:
#Use PySpark SQL
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark.sql.functions as fn

#other modules
import re
import os
import numpy as np
#Saving/reading data to/from disk
import dill
#python dataframe module
import pandas as pd
#Plotting
from matplotlib import pyplot as plt
#Geo-data plotting
import geojson
import folium

In [ ]:
#Set up Spark
#Create a SparkContext
#sc.stop()
sc = SparkContext("local[*]", "pyspark_df")

#Create a SQLContext
sqlContext = SQLContext(sc)


In [ ]:
#Functions
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

# Code for part 1 starts here. 
I read the records and count the number of prescriptions per month for each practice. After that the data were binned and a k-means clustering was performed to group and label the practices. When I have grouped the practices by how many HCPs each one has, the data was plotted with respect to each practice's geolocation and compared to a few geolocation related factors that are publically available.

In [ ]:
#Read 18-month records during 201702-201807 using Spark
df_grouped = []
for n in [201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807]:
    records = sc.textFile(localpath('data/PDPI/'+str(n)+'/'))
    df_rec = records.map(lambda line: line.split(','))\
                    .filter(lambda u: u[1] != 'PCT')\
                    .map(lambda u: (u[2],float(u[6]),int(u[9]))).toDF()
    df_rec = df_rec.withColumnRenamed("_1", "Practice_Code")\
                   .withColumnRenamed("_2", "NIC")\
                   .withColumnRenamed("_3", "Date")
    #print(df_rec.head())
    if df_grouped == []:
        df_grouped = df_rec.groupBy(['Practice_Code','Date'])\
                           .agg(fn.count('NIC').alias('Count'),fn.sum('NIC').alias('NIC_sum'))
    else:
        df_grouped = df_grouped.union(\
                                      df_rec.groupBy(['Practice_Code','Date'])\
                                     .agg(fn.count('NIC').alias('Count'),fn.sum('NIC').alias('NIC_sum'))\
                                     )

#dill is an advanced version of pickle. Allows easy writing/reading data to/from disk.
#Except for gathering info from the records, all dataframe manipulations were done using Pandas
dill.dump(df_grouped.toPandas(), open('grouped_records_cost.pkd', 'wb'))

In [ ]:
#Filter out the dummy practices
df_grouped = dill.load(open('grouped_records_cost.pkd', 'rb'))
df_grouped.loc[df_grouped['Practice_Code'].apply(lambda s: 'Y' not in s)]
dill.dump(df_grouped,open('grouped_records_cost.pkd', 'wb'))

In [ ]:
#Compute the statistics with a GroupBy.
df_grouped = dill.load(open('grouped_records_cost.pkd', 'rb'))
df_stats_all = df_grouped.groupby('Practice_Code')['Count','NIC_sum']\
                .agg([np.median,np.var]).dropna()
#Rename the columns
df_stats_all.columns = [' '.join(col).strip() for col in df_stats_all.columns.values]
#df_stats_all

Histogram the data and plot using matplotlib.

In [ ]:
median = df_stats_all['NIC_sum median'].tolist()

fig,ax = plt.subplots()
ax.hist(np.asarray(median),bins=np.arange(50)*3e5/50)
#ax.set_ylim((0,500))
ax.set_xlabel('Sum(NIC) per month \n (median in 18 months)')
ax.set_ylabel('Number of practices')
fig.tight_layout()
fig.savefig('hist1_1.pdf',format='pdf')

In [ ]:
median = df_stats_all['Count median'].tolist()

fig,ax = plt.subplots()
ax.hist(np.asarray(median),bins=np.arange(50)*50)
#ax.set_ylim((0,500))
ax.set_xlabel('#Prescriptions per month \n (median in 18 months)')
ax.set_ylabel('Number of practices')
fig.tight_layout()
fig.savefig('hist1.pdf',format='pdf')

Find out how long a practice has been around and group the data accordingly. Make histograms.

In [ ]:
practices = sc.textFile(localpath('data/ADDR/'))

df_prac_age = practices.map(lambda line: line.split(','))\
              .map(lambda u: (u[0].strip(),u[1].strip()))\
              .filter(lambda v: 'Y' not in v[1])\
              .toDF()\
              .withColumnRenamed('_1','Date').withColumnRenamed('_2','Practice_Code')
df_prac_age = df_prac_age.groupBy('Practice_Code').count()\
                           .withColumnRenamed('count','Months')

dill.dump(df_prac_age.toPandas(), open('df_prac_age.pkd', 'wb'))

y = [row.Months for row in df_prac_age.collect()]

In [ ]:
fig,ax = plt.subplots()
ax.hist(np.asarray(y),bins=np.arange(20)*5)
ax.set_xlabel('Number of months on record since 2011/08')
ax.set_ylabel('Number of practices')
fig.tight_layout()
fig.savefig('hist1_2.pdf',format='pdf')

In [ ]:
#Group the data by how long the practice has been around
df_prac_age = dill.load(open('df_prac_age.pkd', 'rb'))

df = df_stats_all.join(df_prac_age.set_index('Practice_Code'))
group1 = df.loc[df['Months'] > 80]
group2 = df.loc[df['Months'] <= 80]

median1 = group1['NIC_sum median'].tolist()
median2 = group2['NIC_sum median'].tolist()


In [ ]:
fig,ax = plt.subplots()
ax.hist(np.asarray(median1),bins=np.arange(50)*3e5/50,alpha=0.5,label='> 80 months')
ax.hist(np.asarray(median2),bins=np.arange(50)*3e5/50,alpha=0.5,label='<= 80 months')
ax.legend()
ax.set_xlabel('Sum(NIC) per month \n (median during 2017/02-2018/07)')
ax.set_ylabel('Number of practices')
ax.set_title('Group the data by \n how many months the practice has on record')
fig.tight_layout()
fig.savefig('hist1_3.pdf',format='pdf')

Scatter plot a sample of the data with labels.

In [ ]:
df_sample_ngp = pd.read_csv('df_sample_ngp.csv').dropna()

fig,ax = plt.subplots()
for i in range(1,5):
    df = df_sample_ngp.loc[df_sample_ngp['nGP']==i]
    s = 'nGP = '+str(i)
    median = np.asarray(df['Count median'].tolist())
    var = np.asarray(df['Count var'].tolist())
    ax.plot(median,var,'o',label=s)
df = df_sample_ngp.loc[df_sample_ngp['nGP']>=5.]
s = 'nGP >= 5'
median = np.asarray(df['Count median'].tolist())
var = np.asarray(df['Count var'].tolist())
ax.plot(median,var,'o',label=s)

#ax.set_xlim((300,1900))
ax.legend()
ax.set_xlabel('Median #prescriptions')
ax.set_ylabel('Var #prescriptions')
fig.savefig('scatter1.pdf',format='pdf')

K-means. Filter the data first.

In [ ]:
#Make sure all entry has 12-month worth of data
df_prac_age = dill.load(open('df_prac_age.pkd', 'rb'))

filter2 = pd.DataFrame()
filter2['Months'] = df_grouped.groupby('Practice_Code')['Date'].count()
filter2 = filter2.loc[filter2.Months ==18]

#Join all filters
df = df_stats_all.join(filter2,how='inner')
df = df.drop(columns=['Months'])
filtered_data = df.join(df_prac_age.set_index('Practice_Code'),how='inner').drop(columns=['Months'])

Perform clustering analysis using scikit-learn's k-means class

In [ ]:
from sklearn.cluster import KMeans

#Somewhat arbitary cut
sample = filtered_data.loc[filtered_data['Count median'] >300]
sample = sample.loc[sample['Count var'] < 6000].sort_values('Count var')
y = sample['Count var'].tolist()

estimator = KMeans(n_clusters=5).fit(np.asarray(y).reshape(-1, 1))
print(estimator.cluster_centers_)

The labels generated are not ordered, so relabel them. Plot the histograms with labels.

In [ ]:
keys = sample.index
dict_label = {}
for i in range(len(keys)):
    dict_label[keys[i]] = int(estimator.labels_[i])

df_labels = pd.DataFrame.from_dict(dict_label,orient='index',columns=['Label'])
kmeans = sample.join(df_labels)

#A dictionary for which kmeans label correspond to how many GPs
dict_kcenter = {0:2,1:4,2:1,3:5,4:3}
kmeans['nGP'] = kmeans['Label'].apply(lambda n: dict_kcenter[n])

dill.dump(kmeans, open('df_kmeans.pkd', 'wb'))

In [ ]:
fig,ax = plt.subplots()
for i in range(5):
    ki = kmeans.loc[kmeans['nGP'] == i+1]
    var = ki['Count var'].tolist()
    s = '#HCP = '+str(i+1)
    if i+1 == 5:
        s = s+' or more'
    ax.hist(np.asarray(var),bins=np.arange(60)*100,stacked=True,alpha=0.5,label=s)
ax.legend()
ax.set_xlabel('Variation of #prescriptions per month \n (during 2017/02-2018/07)')
ax.set_ylabel('Number of practices')
ax.set_title('Group by how many HCPs the practice has')
fig.tight_layout()
fig.savefig('hist1_4.pdf',format='pdf')

In [ ]:
fig,ax = plt.subplots()
for i in range(5):
    ki = kmeans.loc[kmeans['nGP'] == i+1]
    median = ki['NIC_sum median'].tolist()
    s = '#HCP = '+str(i+1)
    if i+1 == 5:
        s = s+' or more'
    ax.hist(np.asarray(median),bins=np.arange(50)*3e5/50,stacked=True,alpha=0.5,label=s)
ax.legend()
ax.set_xlabel('Median of sum(NIC) per month \n (during 2017/02-2018/07)')
ax.set_ylabel('Number of practices')
ax.set_title('Group by how many HCPs the practice has')
fig.tight_layout()
fig.savefig('hist1_5.pdf',format='pdf')

In [ ]:
fig,ax = plt.subplots()
for i in range(4):
    ki = kmeans.loc[kmeans['nGP'] == i+1]
    median = ki['NIC_sum median'].tolist()
    s = '#HCP = '+str(i+1)
    if i+1 == 5:
        s = s+' or more'
    ax.hist(np.asarray(median)/(i+1),bins=np.arange(50)*2.e5/50,stacked=True,alpha=0.5,label=s)
ax.legend()
ax.set_xlabel('Normalized sum(NIC) per month \n (during 2017/02-2018/07)')
ax.set_ylabel('Number of practices')
ax.set_title('Group by how many HCPs the practice has')
fig.tight_layout()
fig.savefig('hist1_5.pdf',format='pdf')

In [ ]:
fig,ax = plt.subplots()
for i in range(4):
    ki = kmeans.loc[kmeans['nGP'] == i+1]
    median = ki['Count median'].tolist()
    s = '#HCP = '+str(i+1)
    if i+1 == 5:
        s = s+' or more'
    ax.hist(np.asarray(median)/(i+1),bins=np.arange(50)*50,stacked=True,alpha=0.5,label=s)
ax.legend()
ax.set_xlabel('Normalized #prescriptions per month \n (during 2017/02-2018/07)')
ax.set_ylabel('Number of practices')
ax.set_title('Group by how many HCPs the practice has')
fig.tight_layout()
fig.savefig('hist5.pdf',format='pdf')

Work on the geo-location data. Plot the data to postcode areas using folium. The generated html files are interactive maps.

In [ ]:
#Read address book of csv's and generate a table for geolocation of practices
#I decided to use the Postcode areas for cleaner information

def find_areacode(string):
    s = ''
    for x in string:
        if x.isalpha():
            s = s+x
        else:
            break
    return s

#Create an RDD for the addr files
practices = sc.textFile(localpath('data/ADDR/'))

#Read lines, extract data and convert to Spark SQL dataframe
df_geo = practices.map(lambda line: line.split(','))\
              .map(lambda u: (int(u[0]),u[1].strip(),find_areacode(u[7])))\
              .distinct().toDF()
df_geo = df_geo.withColumnRenamed("_1", "Date")\
               .withColumnRenamed("_2", "Practice_Code").withColumnRenamed("_3", "Postcode_Area")

#In case of a practice with more than one location, keep the latest one.
lastentry_udf_str = udf(lambda u: u[-1], StringType())#Define the udf
df_geo = df_geo.orderBy(['Date','Practice_Code'])\
               .groupBy('Practice_Code').agg(fn.collect_list('Postcode_Area').alias('list'))
df_geo = df_geo.withColumn('Postcode_Area',lastentry_udf_str(df_geo.list))\
               .select('Practice_Code','Postcode_Area')

#Save to disk
dill.dump(df_geo.toPandas(), open('df_geo.pkd', 'wb'))
#df_geo = dill.load(open('df_geo.pkd', 'rb'))

In [ ]:
kmeans = dill.load(open('df_kmeans.pkd', 'rb'))
kmeans['Normalized NIC sum'] = kmeans['NIC_sum median']/kmeans['nGP']
kmeans['Normalized count'] = kmeans['Count median']//kmeans['nGP']
df_geo = dill.load(open('df_geo.pkd', 'rb'))
kmeans_geo = kmeans.join(df_geo.set_index('Practice_Code'))

kn = []
for i in range(4):
    ki = kmeans_geo.loc[kmeans_geo['nGP']==i+1]
    ki = ki.groupby('Postcode_Area')['NIC_sum median'].agg([np.median,'count'])
    kn.append(ki)

#Read population data from csv
df_pop = pd.read_csv(os.path.join(os.path.abspath(os.path.curdir), 'pop_density.csv'))
def select_area(s):
    return s.partition('-')[0].strip()
df_pop['Postcode_Area'] = df_pop['postcode areas'].apply(select_area)
df_pop = df_pop.drop(columns='postcode areas')

#Read postcode area topology data
post_area = os.path.join(os.path.abspath(os.path.curdir), 'Areas.json')

In [ ]:
df_age = pd.read_csv(os.path.join(os.path.abspath(os.path.curdir), '419412172.csv'))
#df_age.head()
df_age['Postcode_Area'] = df_age['mnemonic'].apply(select_area)
df_age = df_age.drop(columns=['postcode areas','mnemonic'])
df_age['Over 60'] = df_age['Age 60 to 64']+df_age['Age 65 to 74']\
                    +df_age['Age 75 to 84']+df_age['Age 85 to 89']+df_age['Age 90 and over']

In [ ]:
df = kn[0].join(df_pop.set_index('Postcode_Area'))
df['Postcode_Area'] = df.index

#Create a folium map instance and do a choropleth plot
m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    data=df,
    columns=['Postcode_Area','median'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='BuPu',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_median1.html')
m

In [ ]:
df = kmeans_geo.loc[kmeans_geo['nGP'] <=4].copy()
df = df.groupby('Postcode_Area')['Normalized NIC sum'].agg([np.median,'count'])
df = df.loc[df['count'] >15]
df['Postcode_Area'] = df.index

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    data=df,
    columns=['Postcode_Area','median'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='BuPu',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_median_norm.html')

In [ ]:
df = kmeans_geo.loc[kmeans_geo['nGP'] <=4].copy()
df = df.groupby('Postcode_Area')['Normalized count'].agg([np.median,'count'])
df = df.loc[df['count'] >5]
df['Postcode_Area'] = df.index

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    data=df,
    columns=['Postcode_Area','median'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='BuPu',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_cnt_norm.html')

In [ ]:
df = kmeans_geo.loc[kmeans_geo['nGP'] <=4].copy()
df = kmeans_geo.copy()
#df = df.loc[df['Normalized NIC sum'] >1e5]
#df = df.groupby('Postcode_Area')['Normalized NIC sum'].agg([np.median,'count'])
df = df.loc[df['Normalized count']>1200]
df = df.groupby('Postcode_Area')['Normalized count'].agg([np.median,'count'])
df['Postcode_Area'] = df.index

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    data=df,
    columns=['Postcode_Area','count'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='BuPu',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_high_performing.html')


In [ ]:
#df = kmeans_geo.loc[kmeans_geo['nGP'] <=4].copy()
df = kmeans_geo.copy()
df = df.loc[df['Normalized NIC sum'] <2e4]
df = df.groupby('Postcode_Area')['Normalized NIC sum'].agg([np.median,'count'])
df['Postcode_Area'] = df.index

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    data=df,
    columns=['Postcode_Area','count'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='BuPu',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_low_performing.html')

In [ ]:
df = kmeans_geo.loc[kmeans_geo['nGP'] <=4].copy()
df = kmeans_geo.copy()
#df = df.loc[df['Normalized NIC sum'] >1e5]
#df = df.groupby('Postcode_Area')['Normalized NIC sum'].agg([np.median,'count'])
df = df.loc[df['Normalized count']<400]
df = df.groupby('Postcode_Area')['Normalized count'].agg([np.median,'count'])
df['Postcode_Area'] = df.index

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    data=df,
    columns=['Postcode_Area','count'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='BuPu',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_low_performing.html')


In [ ]:
df = df_pop.copy().set_index('Postcode_Area')
df['GP_sum'] = filtered_data.join(df_geo.set_index('Practice_Code'))\
                                    .groupby('Postcode_Area').apply(len)
df['Postcode_Area'] = df.index
df['Popolation_per_Practice'] = df['Population']/df['GP_sum']
#df = df.drop(index=['AL'])

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    name='Pop_per_Practice',
    data=df,
    columns=['Postcode_Area','Popolation_per_Practice'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='PiYG',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
m.save('map_density.html')

In [ ]:
df = df_age.copy().set_index('Postcode_Area')
df['senior_frac'] = df['Over 60']/df['All usual residents']
#df['GP_sum'] = filtered_data.join(df_geo.set_index('Practice_Code'))\
#                                    .groupby('Postcode_Area').apply(len)
df['Postcode_Area'] = df.index
#df['Popolation_per_Practice'] = df['Over 60']/df['GP_sum']
#df = df.drop(index=['AL'])

m = folium.Map(location=[53, -2], zoom_start=6)

folium.Choropleth(
    geo_data=post_area,
    name='Pop_per_Practice',
    data=df,
    columns=['Postcode_Area','senior_frac'],
    key_on='feature.properties.name',
    bins=8,
    fill_color='PiYG',
    fill_opacity=.9,
    line_opacity=0.5,
    nan_fill_opacity=0.
).add_to(m)

folium.LayerControl().add_to(m)
#m.save('map_senior_density.html')
m

# Code for part 2 starts here.
The records are gone through again using Spark, this time looking for pradax prescriptions and its main competitors.

In [ ]:
bnflist = ['0208020X0', '0208020Y0', '0208020Z0']
grouped = []
for n in [201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807]:
    records = sc.textFile(localpath('data/PDPI/'+str(n)+'/'))
    df_rec = records.map(lambda line: line.split(','))\
                    .filter(lambda u: u[1] != 'PCT')\
                    .map(lambda u: (u[2],u[3][:9],int(u[9])))\
                    .filter(lambda v: v[1] in bnflist)\
                    .toDF()
    df_rec = df_rec.withColumnRenamed("_1", "Practice_Code")\
               .withColumnRenamed("_2", "BNF")\
               .withColumnRenamed("_3", "Date")
    print(df_rec.head())
    if grouped == []:
        grouped = df_rec
    else:
        grouped = grouped.union(df_rec)

dill.dump(grouped.toPandas(), open('pradax_records.pkd', 'wb'))

In [ ]:
#Compute #prescription for Pradax and competitors per month for each practice during 201702-201807
df_meds = dill.load(open('pradax_records.pkd', 'rb'))

dict_meds = {'0208020X0':0, '0208020Y0':1, '0208020Z0':2}
dict_labels = {0:'Pradax',1:'Xarelto',2:'Eliquis'}

df_meds['Label'] = df_meds['BNF'].apply(lambda u: dict_meds[u])

df_meds_cnt = pd.DataFrame()
for i in range(3):
    df = pd.DataFrame()
    df[dict_labels[i]] = df_meds.loc[df_meds['Label']==i].groupby('Practice_Code')['BNF'].count()
    if len(df_meds_cnt) == 0:
        df_meds_cnt = df
    else:
        df_meds_cnt = df_meds_cnt.join(df,how='outer')

Compute the fraction of Pradax each practice prescribed in 18 months. 

In [ ]:
#Overall fraction
fractions = df_meds.groupby('BNF')['Practice_Code'].count().tolist()
fractions = np.asarray(fractions)/np.sum(fractions)

#Plot into a pie chart
fig,ax = plt.subplots(figsize=(6, 3), subplot_kw=dict(aspect="equal"))
ax.pie(fractions,explode=[0.1,0.,0.],labels=('Pradax', 'Xarelto', 'Eliquis'),autopct='%1.1f%%',shadow=True)
fig.savefig('pie1.pdf',format='pdf')

In [ ]:
#Fractions for individual practices
df_meds_cnt = df_meds_cnt.fillna(0)
df_meds_cnt['Total'] = df_meds_cnt.sum(axis=1)

df_meds_frac = pd.DataFrame()
df_meds_frac['Fraction'] = df_meds_cnt['Pradax']/df_meds_cnt['Total']
df_meds_frac['Total'] = df_meds_cnt['Total']/18.

In [ ]:
#A weighted histogram

x = df_meds_frac['Fraction'].tolist()
w = df_meds_frac['Total'].tolist()

fig,ax = plt.subplots()
ax.hist(np.asarray(x),bins=np.arange(50)/50.,weights=w)
ax.set_xlim((-0.05,0.55))
ax.vlines(0.2313,0,5000,linestyles='dashed')
#ax.legend()
ax.set_xlabel('Fraction of Pradax among competitors \n (2017/02-2018/07)')
ax.set_ylabel('Number of practices')
#ax.set_title('Group the data by \n how many months the practice has on record')
fig.tight_layout()
fig.savefig('hist2_2.pdf',format='pdf')

Define a new visiting scheme and compute #visits per week for each practice

In [ ]:
df_meds_frac['weight'] = (1.- df_meds_frac['Fraction'])*df_meds_frac['Total']+2.
df_meds_frac['visits_per_week'] = df_meds_frac['weight'] / sum(df_meds_frac['weight']) * len(df_meds_frac) * 0.7
df_meds_frac['weeks_per_visit'] = 1./df_meds_frac['visits_per_week']